In [1]:
output_path = "/home/jovyan/work/filtered_df_output.parquet"

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("E-commerce Amazing Analysis") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.sql.shuffle.partitions", "2") \
    .config("spark.driver.cores", "1") \
    .config("spark.executor.cores", "1") \
    .getOrCreate()

from pyspark.sql.functions import row_number, col, split, month, year, datediff, sum as _sum,\
    avg as _avg, count as _count, max as _max, current_date, round as _round, coalesce
from pyspark.sql.window import Window

# Lire le fichier Parquet
filtered_df = spark.read.parquet(output_path)

# Ajouter les colonnes 'month' et 'year'
filtered_df = filtered_df.withColumn("month", month(col("event_time"))) \
                         .withColumn("year", year(col("event_time")))

# Filtrer les événements d'achat
purchase_df = filtered_df.filter(col("event_type") == "purchase")

# Extraire les paires uniques category_id et category_code
category_mapping_df = purchase_df.select("category_id", "category_code").distinct()

# Sauvegarder ce mapping dans un fichier Parquet pour une utilisation ultérieure
mapping_output_path = "/home/jovyan/work/category_mapping.parquet"
category_mapping_df.write.mode("overwrite").parquet(mapping_output_path)

# Charger le mapping depuis le fichier Parquet
category_mapping_df = spark.read.parquet(mapping_output_path)

# Renommer la colonne 'category_code' dans le DataFrame de mapping pour éviter l'ambiguïté
category_mapping_df = category_mapping_df.withColumnRenamed("category_code", "mapped_category_code")

# Joindre purchase_df avec category_mapping_df pour ajouter la colonne 'mapped_category_code'
purchase_df_with_mapping = purchase_df.join(category_mapping_df, on="category_id", how="left")

# Remplacer les valeurs NULL dans 'category_code' par les valeurs correspondantes de la jointure
purchase_df = purchase_df_with_mapping.withColumn(
    "category_code",
    coalesce(purchase_df_with_mapping["category_code"], purchase_df_with_mapping["mapped_category_code"])
)

# Calcul des variables explicatives supplémentaires
# 1. Nombre de vues par utilisateur
number_of_views = filtered_df.filter(col("event_type") == "view") \
                             .groupBy("user_id") \
                             .agg(_count("event_type").alias("number_of_views"))

# 2. Nombre de produits ajoutés au panier par utilisateur
number_of_carts = filtered_df.filter(col("event_type") == "cart") \
                             .groupBy("user_id") \
                             .agg(_count("event_type").alias("number_of_carts"))

# 3. Nombre total d'achats précédents par utilisateur
previous_purchases = filtered_df.filter(col("event_type") == "purchase") \
                                .groupBy("user_id") \
                                .agg(_count("event_type").alias("user_previous_purchases"))

# 4. Valeur moyenne des achats précédents par utilisateur
average_purchase_value = filtered_df.filter(col("event_type") == "purchase") \
                                    .groupBy("user_id") \
                                    .agg(_round(_avg("price"), 2).alias("user_average_purchase_value"))

# 5. Temps écoulé depuis le dernier achat
last_purchase_date = filtered_df.filter(col("event_type") == "purchase") \
                                .groupBy("user_id") \
                                .agg(_max("event_time").alias("last_purchase_date"))

days_since_last_purchase = last_purchase_date.withColumn("days_since_last_purchase", 
                                                         datediff(current_date(), col("last_purchase_date")))

# 6. Nombre de produits ajoutés au panier mais non achetés (abandons de panier)
cart_abandonments = filtered_df.filter(col("event_type") == "cart") \
                               .groupBy("user_id", "product_id") \
                               .agg(_count("event_type").alias("cart_count")) \
                               .join(purchase_df.groupBy("user_id", "product_id").agg(_count("event_type").alias("purchase_count")),
                                     on=["user_id", "product_id"], how="left") \
                               .withColumn("purchase_count", col("purchase_count").cast("int")) \
                               .na.fill(0) \
                               .filter(col("purchase_count") == 0) \
                               .groupBy("user_id") \
                               .agg(_count("product_id").alias("cart_abandonments"))

# 7. Valeur totale des achats par utilisateur
total_purchase_value = filtered_df.filter(col("event_type") == "purchase") \
                                  .groupBy("user_id") \
                                  .agg(_round(_sum("price"), 2).alias("total_purchase_value"))

# 8. Nombre total de sessions par utilisateur
number_of_sessions = filtered_df.groupBy("user_id") \
                                .agg(_count("user_session").alias("number_of_sessions"))

# Calcul des agrégations nécessaires
# count_products_per_month = purchase_df.groupBy("user_id", "year", "month") \
#                                       .agg(_count("product_id").alias("count_products_per_month"))
# sum_products_per_month = purchase_df.groupBy("user_id", "year", "month") \
#                                     .agg(_round(_sum("price"), 2).alias("sum_products_per_month"))
# avg_price_per_month = purchase_df.groupBy("user_id", "year", "month", "category_code") \
#                                  .agg(_round(_avg("price"), 2).alias("avg_price_per_month"))

# Ajouter 'category_code' à count_products_per_month et sum_products_per_month pour les jointures ultérieures
# count_products_per_month = count_products_per_month.join(purchase_df.select("user_id", "year", "month", "category_code").distinct(), 
#                                                          on=["user_id", "year", "month"], how="left")
# sum_products_per_month = sum_products_per_month.join(purchase_df.select("user_id", "year", "month", "category_code").distinct(), 
#                                                      on=["user_id", "year", "month"], how="left")

# Joindre les DataFrames pour créer le DataFrame final
# final_df = count_products_per_month.join(sum_products_per_month, ["user_id", "year", "month", "category_code"]) \
#                                    .join(avg_price_per_month, ["user_id", "year", "month", "category_code"]) \
#                                    .join(number_of_views, "user_id", "left") \
#                                    .join(number_of_carts, "user_id", "left") \
#                                    .join(previous_purchases, "user_id", "left") \
#                                    .join(average_purchase_value, "user_id", "left") \
#                                    .join(days_since_last_purchase, "user_id", "left") \
#                                    .join(cart_abandonments, "user_id", "left") \
#                                    .join(total_purchase_value, "user_id", "left") \
#                                    .join(number_of_sessions, "user_id", "left")

# Joindre les DataFrames sans les variables temporelles pour créer le DataFrame final
final_df=number_of_views.join(number_of_carts, "user_id", "left") \
                        .join(previous_purchases, "user_id", "left") \
                        .join(average_purchase_value, "user_id", "left") \
                        .join(days_since_last_purchase, "user_id", "left") \
                        .join(cart_abandonments, "user_id", "left") \
                        .join(total_purchase_value, "user_id", "left") \
                        .join(number_of_sessions, "user_id", "left")


# Remplacer les valeurs NULL par des valeurs par défaut si nécessaire
final_df = final_df.fillna({
    "number_of_views": 0,
    "number_of_carts": 0,
    "user_previous_purchases": 0,
    "user_average_purchase_value": 0.0,
    "days_since_last_purchase": 9999,  # Utiliser une valeur par défaut pour indiquer une absence de précédent achat
    "cart_abandonments": 0,
    "total_purchase_value": 0.0,
    "number_of_sessions": 0
})

# # Catégorie de produit la plus achetée par utilisateur et par mois
# window_spec = Window.partitionBy("user_id", "year", "month").orderBy(col("count_products_per_month").desc())
# most_purchased_category_df = count_products_per_month.withColumn("rank", row_number().over(window_spec)) \
#                                                      .filter(col("rank") == 1) \
#                                                      .select("user_id", "year", "month", "category_code") \
#                                                      .withColumnRenamed("category_code", "most_purchased_category")

# Catégorie de produit la plus achetée par utilisateur

window_spec = Window.partitionBy("user_id", "year", "month").orderBy(col("count_products_per_month").desc())
most_purchased_category_df = count_products_per_month.withColumn("rank", row_number().over(window_spec)) \
                                                     .filter(col("rank") == 1) \
                                                     .select("user_id", "year", "month", "category_code") \
                                                     .withColumnRenamed("category_code", "most_purchased_category")

# # Joindre la catégorie de produit la plus achetée au DataFrame final
# final_df = final_df.join(most_purchased_category_df, ["user_id", "year", "month"], "left")

# # Calculer le nombre total de produits achetés par catégorie et par mois
# total_products_per_category_month = purchase_df.groupBy("category_code", "year", "month") \
#                                                .agg(_count("product_id").alias("total_products_per_category_month"))

# # Joindre l'information du nombre total de produits achetés par catégorie et par mois au DataFrame final
# final_df = final_df.join(total_products_per_category_month, ["category_code", "year", "month"], "left")




Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/23 10:08:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/23 10:08:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
final_df = final_df.select(
    "user_id",
    "year",
    "month",
    "category_code",
    "most_purchased_category",
    # "count_products_per_month",
    # "sum_products_per_month",
    # "avg_price_per_month",
    # "total_products_per_category_month",
    "number_of_views",
    "number_of_carts",
    "cart_abandonments",
    "user_previous_purchases",
    "user_average_purchase_value",
    "total_purchase_value",
    "last_purchase_date",
    "days_since_last_purchase",
    "number_of_sessions"
)

In [3]:
# Sauvegarder le résultat dans un nouveau fichier Parquet
final_df.write.mode("overwrite").parquet("/home/jovyan/work/final_combined_df.parquet")



In [4]:
# Afficher quelques lignes sans troncature
final_combined_df = spark.read.parquet("/home/jovyan/work/final_combined_df.parquet")
final_combined_df.show(truncate=False)

+---------+----+-----+------------------------------+------------------------------+------------------------+----------------------+-------------------+---------------------------------+---------------+---------------+-----------------+-----------------------+---------------------------+--------------------+-------------------+------------------------+------------------+
|user_id  |year|month|category_code                 |most_purchased_category       |count_products_per_month|sum_products_per_month|avg_price_per_month|total_products_per_category_month|number_of_views|number_of_carts|cart_abandonments|user_previous_purchases|user_average_purchase_value|total_purchase_value|last_purchase_date |days_since_last_purchase|number_of_sessions|
+---------+----+-----+------------------------------+------------------------------+------------------------+----------------------+-------------------+---------------------------------+---------------+---------------+-----------------+----------------

In [ ]:
# Conversion des colonnes nécessaires en types appropriés
purchase_df = purchase_df.withColumn("price", col("price").cast("float"))
purchase_df = purchase_df.withColumn("year", col("year").cast("int"))
purchase_df = purchase_df.withColumn("month", col("month").cast("int"))

In [ ]:
# Arrêter la session Spark
spark.stop(